# Data Collection Phase

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import shap


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B111F8D5-6AC6-3245-A6B5-94693F6992AB> /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/O

In [ ]:
from datapull import fetch_datasets

fetch_datasets(payload={
    "locations": ["132"],
    "dateType": "D",
    "startDate": "2013-1-1",
    "endDate": "2025-11-15"
}, file_path="data/air_quality_2013-2025.csv")

Processed 1/25: Carbon Dioxide
Processed 2/25: Methane
Processed 3/25: Sulfur Hexafluoride
Processed 4/25: Nitrous Oxide
Processed 5/25: Surface Ozone
Processed 6/25: Sulfur Dioxide
Processed 7/25: Carbon Monoxide
Processed 8/25: Nitrogen Oxides
Processed 9/25: PM10
Processed 10/25: Condensation Particle Count
Processed 11/25: Scattering Coefficient
Processed 12/25: Absorption Coefficient
Processed 13/25: Aerosol Optical Depth
Processed 14/25: Fine Particle Size Distribution
Processed 15/25: Particle Size Distribution
Processed 16/25: UV-A Radiation
Processed 17/25: UV-B Radiation
Processed 18/25: Direct Solar Radiation
Processed 19/25: Diffuse Solar Radiation
Processed 20/25: Solar Upward Radiation
Processed 21/25: Solar Downward Radiation
Processed 22/25: Terrestrial Upward Radiation
Processed 23/25: Terrestrial Downward Radiation
Processed 24/25: Net Radiation
Processed 25/25: Total Ozone Column

------------------------------------------
✅ Runtime: 33679.7 ms | 💾 Mem: 262.25 MB (Δ 

In [ ]:
df = pd.read_csv('data/air_quality_2013-2025.csv')
df.isna().sum().sort_values()

qcDone_y        0
OBS_DATE        0
uvmax_x       102
uvsum_x       102
CH4_conff     111
             ... 
uvsumb_y     4687
uvsumb_x     4687
CH4_fdata    4701
N2O_fdata    4701
CO2_fdata    4701
Length: 235, dtype: int64


------------------------------------------
✅ Runtime: 91.7 ms | 💾 Mem: 247.83 MB (Δ +36.17 MB) | ⚙️ CPU: 30.2%


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701 entries, 0 to 4700
Columns: 235 entries, CO2_fdata to ozone
dtypes: float64(233), int64(1), object(1)
memory usage: 8.4+ MB

------------------------------------------
✅ Runtime: 21.5 ms | 💾 Mem: 237.16 MB (Δ -10.67 MB) | ⚙️ CPU: 45.7%


In [ ]:
df.describe()

,CO2_fdata,CO2_confs,CO2_conff,CO2_sdata,CH4_conff,CH4_confs,CH4_fdata,CH4_sdata,SF6_sdata,SF6_confs,...,uvmaxb_y,uvmax_y,sdt,sdf,sup,sdn,lup,ldn,net,ozone
count,0.0,3045.000000,4585.000000,19.000000,4590.000000,2757.000000,0.0,19.000000,86.000000,2651.000000,...,14.000000,4563.000000,3664.000000,3896.000000,4120.000000,3705.000000,3936.000000,3950.000000,3744.000000,3539.000000
mean,NaN,418.834436,417.327332,442.422688,1975.012686,1979.077587,NaN,2101.563521,14.387902,10.312093,...,0.063525,0.122586,153.210948,80.106216,35.580973,177.994409,376.385090,308.367168,80.056221,325.167240
std,NaN,10.421129,10.399041,9.912959,46.632456,52.430631,NaN,61.498409,2.897112,1.330344,...,0.005403,0.065025,114.293886,40.997501,16.632777,88.315756,52.157254,57.209343,59.950700,37.629232
min,NaN,385.531316,393.529951,430.808792,1816.114479,1790.113065,NaN,2042.147109,12.285302,7.838400,...,0.051450,0.005000,0.000000,0.000000,0.912500,7.568318,229.599116,157.366667,-36.401942,245.210000
25%,NaN,410.998480,409.645004,435.263229,1947.798669,1947.619720,NaN,2050.499125,12.469728,9.349047,...,0.062650,0.063886,42.898329,48.601877,22.645078,106.016667,333.546698,263.391571,27.476739,296.208841
50%,NaN,418.776154,417.200959,440.052895,1969.127701,1976.095498,NaN,2082.610557,13.578679,10.052645,...,0.064050,0.116000,147.689450,73.197917,36.488851,173.268750,375.081958,303.879167,70.775000,318.050000
75%,NaN,426.939177,425.947454,447.404465,2014.220764,2021.850923,NaN,2118.716652,14.975522,11.653041,...,0.066762,0.174000,246.514420,110.022742,49.196527,254.425242,419.444792,353.100578,130.724165,350.545300
max,NaN,444.230546,438.968422,467.420569,2068.679768,2162.085114,NaN,2256.504054,29.788935,13.238464,...,0.071400,0.331100,455.320833,217.112500,88.891667,367.215213,527.920833,440.907099,226.676301,473.120000



------------------------------------------
✅ Runtime: 172.1 ms | 💾 Mem: 221.52 MB (Δ -15.64 MB) | ⚙️ CPU: 27.9%


In [ ]:
df.isna().sum()

In [ ]:
df = df.drop_duplicates()
df.isna().sum()

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns
plt.figure(figsize=(15, 8))
df[numeric_cols].boxplot(rot=90)
plt.title("Outlier Detection")

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap="Blues")
plt.title("Correlation Heatmap")

In [ ]:
df["precipitation"].describe()
df["precipitation"].unique()

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns
plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap="Blues")
plt.title("Correlation Heatmap")

In [ ]:
for col in df.columns:
    if np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = df[col].astype(int) // 10**9
from sklearn.preprocessing import LabelEncoder

for col in df.columns:
    if df[col].dtype == "object":
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

In [ ]:
target = "temperature_2m"
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8
)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("R2:", r2_score(y_test, y_pred))
rmse = mean_squared_error(y_test, y_pred) ** 0.5
print("RMSE:", rmse)

In [ ]:
# Feature importance
plt.figure(figsize=(10,6))
plt.barh(X.columns, model.feature_importances_)
plt.title("XGBoost Feature Importance")

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred)
min_val = min(min(y_test), min(y_pred))
max_val = max(max(y_test), max(y_pred))
plt.plot([min_val, max_val], [min_val, max_val])
plt.xlabel("Actual Temperature")
plt.ylabel("Predicted Temperature")
plt.title("Predicted vs Actual Temperature")

In [ ]:
import xgboost as xgb
xgb.plot_importance(model)

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance.head(10)) # <--- most important 10 features
time_importance_pct = (feature_importance.iloc[0]['importance'] / feature_importance['importance'].sum()) * 100
print(f"\nTime feature accounts for {time_importance_pct:.1f}% of total feature importance")